In [24]:
import re
import sys
import os
import warnings
from pdfminer.high_level import extract_text, extract_text_to_fp
from PyPDF2 import PdfFileReader
import tabula
from country_list import countries_for_language as countries_in
import pandas as pd
import numpy as np
import requests
import camelot
from pdfminer.pdfparser import PDFParser
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfinterp import resolve1
if sys.version_info > (3, 0):
    from io import StringIO
else:
    from io import ByteIO as StringIO
from pdfminer.layout import LAParams


warnings.filterwarnings('ignore')
path = "G:\\upwork-items\\pdf-parsing"
os.chdir(os.path.join(path))
pd.set_option("display.max_rows", 2000)
pd.set_option("display.max_columns", 2000)
#================================new cell====================================
f = open('currencies.txt')
currencies = f.readline().split()
f.close()

COUNTRIES=list(countries_in('en'))
COUNTRIES = [x[1] for x in COUNTRIES ]
COUNTRIES.extend(['United States of America','Virgin Islands, British',
                  'Virgin Islands, US','Virgin Islands, U.S.','Convertible Bonds',
                 "Equities","Bonds","Options","Financial instruments","Investment funds (UCITS)","Investment funds",
                  "Long Positions",
                 ])
#================================new cell====================================
pdf_links_df = pd.read_csv('banque.csv')
pdf_links_df = pdf_links_df.sort_values(['name'])
print(pdf_links_df.to_string())
links = set(pdf_links_df['pdf_url'])
links.discard({'nan', 'annual_report_does_not_exists'})
print(len(links))
#================================new cell====================================

# import requests
# links_to_pdf_map = {}
# num = 1
# print('Downloading ' + str(len(links))+' files...')
# for link in links:
#     if link in ['nan','annual_report_does_not_exists']:
#         continue
#     response = requests.get(link)
#     print('banque'+ str(num) +'.pdf')
#     links_to_pdf_map[link] = 'banque'+ str(num) +'.pdf'
#     with open('banque'+ str(num) +'.pdf','wb') as f:
#         f.write(response.content)
#     num+=1

# del num
#del response

'''
# when you dont want to download, only test with existing downloaded PDFs
num=1
links_to_pdf_map = {}

for link in links:
    if link in ['nan','annual_report_does_not_exists']:
        continue
    links_to_pdf_map[link] = 'banque'+ str(num) +'.pdf'
    num+=1
del num
'''

#================================new cell====================================
links_to_pdf_map[np.nan]= np.nan
links_to_pdf_map['annual_report_does_not_exists'] = np.nan
# pdf_links_df['files'] = pdf_links_df['pdf_url'].apply(lambda x: links_to_pdf_map[x])
# pdf_links_df.to_csv('banque.csv',index = False)
#================================new cell====================================

def parse_type1(pdf,read_file,website,pdf_url):
    print('at parse type 1...')
    print(f'file=> {read_file} ; pages => {pdf.getNumPages() } ; ')
    text = 'Désignation des valeurs'
    text2 = 'Désignation des valeurs Quantité Cours' # for another parser type
    #pages =[i+1 for i in range(pdf.getNumPages()) if text in pdf.getPage(i).extractText().replace('\n','')]
    pages = []
    pages2 = []
    for i in range(pdf.getNumPages()):
        #print(f'now scanning page=> {i}')
        text_of_page = extract_text(read_file, page_numbers=[i])
        if all([txt in text_of_page for txt in text.split()]):
            pages.append(i+1)
    for i in range(pdf.getNumPages()):
        #print(f'now scanning page=> {i}')
        text_of_page = extract_text(read_file, page_numbers=[i])
        if all([txt in text_of_page for txt in text2.split()]):
            pages2.append(i+1)
    
    print(f'pages where "Désignation des valeurs" was found => {pages}')
    print(f'pages2 where "Désignation des valeurs Quantité Cours" was found => {pages2}')
    if len(pages) > 0 and len(pages2) == 0:
        print("At pagesssssss")
        tables = tabula.read_pdf(read_file, pages=pages, columns=[345,364,434,514,552], silent=True, guess=False, stream=True)
        for i in range(len(tables)):
            tables[i].columns = ['holding_name','currency','_1','market_value','net_assets']
            tables[i].drop(columns = ['_1'], inplace=True)
    if len(pages2) > 0:
        print("At pagessss2222")
        tables = tabula.read_pdf(read_file, pages=pages2, columns=[308,369,421,448,516,560], silent=True, guess=False, stream=True)
        for i in range(len(tables)):
            tables[i].columns = ['holding_name','_1','_2','currency','market_value','net_assets']
            tables[i].drop(columns = ['_1','_2'], inplace=True)
    
    table = pd.concat(tables, ignore_index = True)
    table['fund_name_website'] = table['fund_name_report'] = website
    table['pdf_url'] = pdf_url
    table = table[table['currency'].isin(currencies)]
    table = table[table['net_assets'].notna()]
    table['market_value'] = table['market_value'].str.replace(',', '.').replace('\s', '', regex=True).replace('[a-zA-Z]', '', regex=True).astype(float)
    table['net_assets'] = table['net_assets'].str.replace(',', '.').replace('\s', '', regex=True).replace('[a-zA-Z]', '', regex=True).astype(float)  
        
    df = table[:]
    funds= set(df['fund_name_website'])
    for x in funds:
        net_assets = float("{:.2f}".format(df[df['fund_name_website']==x]['net_assets'].sum()))
        print('fund name: ',x,' ,    total net_assets: ',net_assets ,',     holdings: ',len(df[df['fund_name_website']==x]),end='\n\n')    
    print(table)
    return table
    
'''
def parse_type2(pdf,read_file,website,pdf_url):
    pdf = PdfFileReader(read_file)
    pdf._override_encryption = True
    pdf._flatten()
    text1 = 'MANDARINE UNIQUE'
    text2 = 'Portefeuille-Titres au'
    file = open('banque4.pdf', 'rb')
    parser = PDFParser(file)
    document = PDFDocument(parser)
    tot_pg = resolve1(document.catalog['Pages'])['Count']
    pages = [i+1 for i in range(tot_pg) if text1 in pdf.getPage(i).extractText().replace('\n','') and text2 in pdf.getPage(i).extractText().replace('\n','')]
    pages.pop(0)
    report = re.search(r'\n(.*)\n',pdf.getPage(pages[0]).extractText()).group(1)  
    tables = tabula.read_pdf(read_file, pages = pages,columns =(76,207,222,271,299,344,476,491,539), silent = True, guess = False,stream = True)
    for i in range(len(tables)):
        df1,df2 = tables[i].iloc[:,:5],tables[i].iloc[:,5:]
        df1.columns = df2.columns = ['_1','holding_name','currency','market_value','net_assets']
        df1.drop(columns = ['_1'],inplace =True)
        df2.drop(columns = ['_1'],inplace = True)
        df = pd.concat([df1,df2], ignore_index = True)
        tables[i] = df
    table = pd.concat(tables, ignore_index= True)
    table['fund_name_website'] = website
    table['fund_name_report'] = report
    table['pdf_url'] = pdf_url
    return table
'''

#================================new cell====================================
# _type = None
# tables = []

# #for idx, row in pdf_links_df.sort_values(by=['files']).iterrows():
# for idx, row in pdf_links_df.iterrows():    
#     website = row['name']
#     read_file = row['files']
#     pdf_url = row['pdf_url']
#     print(type(read_file))
#     #if read_file == 'banque39.pdf':
#     if isinstance(read_file, str):
#         pdf = PdfFileReader(read_file)
#         pdf._override_encryption = True
#         pdf._flatten()
#         tables.append(parse_type1(pdf,read_file,website,pdf_url))
#     else:
#         tables.append(pd.DataFrame([]))
# #print(tables)
# df = pd.concat(tables)
# df.to_csv('banque-result.csv')  

#     if website in pdf.getPage(0).extractText().replace('\n',''):
#         print("TTTYYYPPPEEE 111111111")
#         _type = 1
#     else:
#         print("TTTYYYPPPEEE 222222222")    
#         _type = 2
#     if _type==1:
#         print('parsing type oneeeeeee', read_file, website)
#         tables.append(parse_type1(pdf,read_file,website,pdf_url))
#     else:
#         print('parsing type twooooooo', read_file, website)
#         tables.append(parse_type2(pdf,read_file,website,pdf_url))


                                        name                                                                      pdf_url          files
0                                  ACTYS 1 U  https://www.labanquepostale-am.fr/Lbpam/transversal/pdf/PNM.Par.RA.File.pdf   banque60.pdf
1                                  ACTYS 2 U  https://www.labanquepostale-am.fr/Lbpam/transversal/pdf/PNC.Par.RA.File.pdf    banque2.pdf
2                                  ACTYS 3 U  https://www.labanquepostale-am.fr/Lbpam/transversal/pdf/ATY.Par.RA.File.pdf   banque13.pdf
3                  DECISIEL ISR ACTIONS 70 G  https://www.labanquepostale-am.fr/Lbpam/transversal/pdf/DDQ.Par.RA.File.pdf   banque84.pdf
4        DECISIEL ISR ACTIONS 70 SOLIDAIRE G  https://www.labanquepostale-am.fr/Lbpam/transversal/pdf/DDS.Par.RA.File.pdf   banque99.pdf
5        DECISIEL ISR ACTIONS 70 SOLIDAIRE T  https://www.labanquepostale-am.fr/Lbpam/transversal/pdf/DDS.Par.RA.File.pdf   banque99.pdf
6                  DECISIEL ISR ACTIONS 7

"\ndef parse_type2(pdf,read_file,website,pdf_url):\n    pdf = PdfFileReader(read_file)\n    pdf._override_encryption = True\n    pdf._flatten()\n    text1 = 'MANDARINE UNIQUE'\n    text2 = 'Portefeuille-Titres au'\n    file = open('banque4.pdf', 'rb')\n    parser = PDFParser(file)\n    document = PDFDocument(parser)\n    tot_pg = resolve1(document.catalog['Pages'])['Count']\n    pages = [i+1 for i in range(tot_pg) if text1 in pdf.getPage(i).extractText().replace('\n','') and text2 in pdf.getPage(i).extractText().replace('\n','')]\n    pages.pop(0)\n    report = re.search(r'\n(.*)\n',pdf.getPage(pages[0]).extractText()).group(1)  \n    tables = tabula.read_pdf(read_file, pages = pages,columns =(76,207,222,271,299,344,476,491,539), silent = True, guess = False,stream = True)\n    for i in range(len(tables)):\n        df1,df2 = tables[i].iloc[:,:5],tables[i].iloc[:,5:]\n        df1.columns = df2.columns = ['_1','holding_name','currency','market_value','net_assets']\n        df1.drop(colum

In [25]:
df_actual = pd.read_csv('banque.csv')
df_result = pd.read_csv('banque-result.csv')

names_actual = set(df_actual['name'])
names_result = set(df_result['fund_name_website'])
print(len(names_actual))
print(len(names_result))
missing_list = []
for actual in names_actual:
    if actual not in names_result:
        print(actual)
        missing_list.append(actual)
        
        
missing_pdf_links = pdf_links_df[pdf_links_df['name'].isin(missing_list)]

_type = None
tables = []
#for idx, row in pdf_links_df.sort_values(by=['files']).iterrows():
for idx, row in missing_pdf_links.iterrows():    
    website = row['name']
    read_file = row['files']
    pdf_url = row['pdf_url']
    print(type(read_file))
    #if read_file == 'banque39.pdf':
    if isinstance(read_file, str):
        pdf = PdfFileReader(read_file)
        pdf._override_encryption = True
        pdf._flatten()
        tables.append(parse_type1(pdf,read_file,website,pdf_url))
    else:
        tables.append(pd.DataFrame([]))
#print(tables)
df = pd.concat(tables)
df.to_csv('banque-result-missing.csv')  


173
152
LBPAM 80 PEA HORIZON DECEMBRE 2025 R
SOREA ISR DYNAMIQUE ET SOLIDAIRE E
LBPAM 90 HORIZON DECEMBRE 2027 R
LBPAM ISR ACTIONS VALUE EURO R
FEDERIS ISR EURO L
SOREA ISR ACTIONS EURO F
SOREA ISR ACTIONS EURO E
TOCQUEVILLE CROISSANCE EURO ISR AD
SOREA ISR CROISSANCE E
SOREA ISR MONETAIRE
LBPAM 90 HORIZON JUILLET 2027
SOREA ISR OBLIGATIONS F
SOREA ISR CROISSANCE F
SOREA ISR OBLIGATIONS E
SOREA OBLIGATIONS F
SOREA OBLIGATIONS E
SOREA CROISSANCE E
TOCQUEVILLE CROISSANCE EURO ISR R
TOCQUEVILLE MID CAP EURO ISR L
SOREA CROISSANCE F
SOREA ISR DYNAMIQUE ET SOLIDAIRE F
<class 'str'>
at parse type 1...
file=> banque91.pdf ; pages => 46 ; 
pages where "Désignation des valeurs" was found => [40, 41]
pages2 where "Désignation des valeurs Quantité Cours" was found => [40, 41]
At pagessss2222
fund name:  FEDERIS ISR EURO L  ,    total net_assets:  200.09 ,     holdings:  66

                    holding_name currency  market_value  net_assets  \
4                     ACCIONA SA      EUR  1.040917e+

pages where "Désignation des valeurs" was found => [43, 44]
pages2 where "Désignation des valeurs Quantité Cours" was found => [43, 44]
At pagessss2222
fund name:  LBPAM ISR ACTIONS VALUE EURO R  ,    total net_assets:  200.07 ,     holdings:  68

                    holding_name currency  market_value  net_assets  \
4                 ALLIANZ SE-REG      EUR  3.071754e+07        3.67   
5                      AMUNDI SA      EUR  7.841919e+06        0.94   
6     ANHEUSER-BUSCH INBEV SA/NV      EUR  9.521639e+06        1.14   
7                  ARCELORMITTAL      EUR  1.800261e+07        2.15   
8                         ARKEMA      EUR  1.027537e+07        1.23   
9                        ATOS SE      EUR  7.452500e+06        0.89   
10                        AXA SA      EUR  2.001549e+07        2.39   
11  BANCO BILBAO VIZCAYA ARGENTA      EUR  1.024865e+07        1.22   
12            BANCO SANTANDER SA      EUR  1.799706e+07        2.15   
13                       BASF SE      EUR 

pages where "Désignation des valeurs" was found => [31]
pages2 where "Désignation des valeurs Quantité Cours" was found => [31]
At pagessss2222
fund name:  SOREA CROISSANCE E  ,    total net_assets:  199.75 ,     holdings:  2

                        holding_name currency  market_value  net_assets  \
5   LBPAM DIVERSIFIE FOCUS OBLIG 3-5      EUR   30878177.73       99.75   
10                   TOTAL ACTIF NET      EUR   30955047.08      100.00   

     fund_name_website    fund_name_report  \
5   SOREA CROISSANCE E  SOREA CROISSANCE E   
10  SOREA CROISSANCE E  SOREA CROISSANCE E   

                                              pdf_url  
5   https://www.labanquepostale-am.fr/Lbpam/transv...  
10  https://www.labanquepostale-am.fr/Lbpam/transv...  
<class 'str'>
at parse type 1...
file=> banque46.pdf ; pages => 36 ; 
pages where "Désignation des valeurs" was found => [31]
pages2 where "Désignation des valeurs Quantité Cours" was found => [31]
At pagessss2222
fund name:  SOREA CROISSAN

pages where "Désignation des valeurs" was found => [39]
pages2 where "Désignation des valeurs Quantité Cours" was found => [39]
At pagessss2222
fund name:  SOREA ISR DYNAMIQUE ET SOLIDAIRE F  ,    total net_assets:  200.2 ,     holdings:  9

                        holding_name currency  market_value  net_assets  \
4           FONCIERE HABITAT ACT NOM      EUR    1075960.00        7.29   
8         ALLIANZ VALEURS DURABLES-I      EUR     580442.46        3.93   
9                FEDERIS ISR EURO MH      EUR    8384580.33       56.81   
10       LBPAM ACTIONS INDICE EURO-I      EUR    1759273.46       11.92   
11  LBPAM ISR ACTIONS VALUE EURO 13D      EUR    1756446.00       11.90   
12         LBPAM RESPONSABLE TRESO-I      EUR     352982.35        2.39   
13      LBPAM TOCQU CROIS EU ISR-REA      EUR     298067.00        2.02   
14      SYCOMORE SELECTION RESPONS-I      EUR     581969.10        3.94   
19                   TOTAL ACTIF NET      EUR   14757790.85      100.00   

       

pages where "Désignation des valeurs" was found => [46, 47]
pages2 where "Désignation des valeurs Quantité Cours" was found => [46, 47]
At pagessss2222
fund name:  TOCQUEVILLE CROISSANCE EURO ISR R  ,    total net_assets:  200.08 ,     holdings:  60

                              holding_name currency  market_value  net_assets  \
4                                ADIDAS AG      EUR  2.023069e+07        2.38   
5   AIR LIQUIDE RGS SHS LOYALTY BONUS 2021      EUR  7.383750e+06        0.87   
6                           AIR LIQUIDE SA      EUR  4.353728e+06        0.51   
7                                   ALSTOM      EUR  1.849797e+07        2.18   
8                                AMUNDI SA      EUR  9.614123e+06        1.13   
9                                   ARKEMA      EUR  8.341696e+06        0.98   
10                    ASM INTERNATIONAL NV      EUR  2.483040e+07        2.92   
11                         ASML HOLDING NV      EUR  4.923021e+07        5.79   
12                  

pages where "Désignation des valeurs" was found => [38, 39]
pages2 where "Désignation des valeurs Quantité Cours" was found => [38, 39]
At pagessss2222
fund name:  TOCQUEVILLE MID CAP EURO ISR L  ,    total net_assets:  200.12 ,     holdings:  45

                  holding_name currency  market_value  net_assets  \
4                       ALSTOM      EUR  4.086066e+06        2.95   
5                    AMUNDI SA      EUR  3.233120e+06        2.33   
6         ASM INTERNATIONAL NV      EUR  4.138850e+06        2.98   
7                BANCO BPM SPA      EUR  1.356000e+06        0.98   
8                   BIOMERIEUX      EUR  2.596500e+06        1.87   
9             BUZZI UNICEM SPA      EUR  2.755350e+06        1.99   
10  CARL ZEISS MEDITEC AG - BR      EUR  2.341350e+06        1.69   
11          CELLNEX TELECOM SA      EUR  1.946871e+06        1.40   
12                    DUERR AG      EUR  3.790900e+06        2.73   
13                     EIFFAGE      EUR  3.517280e+06        2